In [6]:
from datetime import timedelta

import requests
import psycopg2
import csv

# Define the path for the input and output files
input_file = '/home/ugoo/airflow/web-server-access-log.txt'
extracted_and_transformed_file = '/home/ugoo/airflow/extracted-data.txt'
output_file = '/home/ugoo/airflow/result.csv'

# Load the data from the weblink and store in a text file
def download():
    global input_file
    url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0250EN-SkillsNetwork/labs/Apache%20Airflow/Build%20a%20DAG%20using%20Airflow/web-server-access-log.txt"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad responses (4xx and 5xx)
        
        with open(input_file, 'w') as file:
            file.write(response.text)
        print(f"File downloaded successfully: {input_file}")
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
        raise

# Extract and transform the data
def extraction():
    global input_file, extracted_and_transformed_file
    try:
        with open(input_file, "r") as infile:
            data = infile.read()
            if not data:
                raise ValueError("Input file is empty.")
            lines = data.splitlines()
        
        with open(extracted_and_transformed_file, "w") as outfile:
            for line in lines:
                field_data = line.strip().split("#")
                if len(field_data) >= 4:
                    timestamp = field_data[0]
                    visitor_id = f"{field_data[3].capitalize()}"
                    outfile.write(timestamp + "#" + visitor_id + "\n")
        print(f"Data extracted and transformed successfully: {extracted_and_transformed_file}")
    
    except IOError as e:
        print(f"Error processing file: {e}")
        raise
    except ValueError as e:
        print(f"ValueError: {e}")
        raise

# Load data into CSV format
def load():
    global extracted_and_transformed_file, output_file
    try:
        with open(extracted_and_transformed_file, 'r') as infile, open(output_file, 'w') as outfile:
            for line in infile:
                row = line.replace("#", ",")
                outfile.write(row + '\n')
        print(f"Data loaded successfully into CSV: {output_file}")
    
    except IOError as e:
        print(f"Error writing to file: {e}")
        raise

# Upload CSV data to PostgreSQL database
def upload():
    global output_file
    try:
        # PostgreSQL connection details
        conn = psycopg2.connect(
            dbname='postgres',
            user='postgres.xoqgqpfqekpszphdvisd',
            password='NKRblwNFomenH15P',
            host='aws-0-eu-central-1.pooler.supabase.com',
            port='6543'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS web_server_logs (
            timestamp TEXT,
            visitor_id TEXT
        );
        """)

        # Load CSV data into PostgreSQL
        with open(output_file, 'r') as f:
            reader = csv.reader(f)
            header = next(reader, None)  # Skip header row
            
            if not header or len(header) != 2:
                raise ValueError("CSV header is missing or has an incorrect number of columns.")
            
            for row in reader:
                # Skip empty rows
                if not row or len(row) != 2:
                    print(f"Skipping invalid row: {row}")
                    continue
                
                # Trim whitespace (if necessary)
                row = [field.strip() for field in row]

                # Insert data into PostgreSQL
                cursor.execute("""
                INSERT INTO web_server_logs (timestamp, visitor_id) VALUES (%s, %s);
                """, row)
        
        conn.commit()
        print(f"Data uploaded successfully to PostgreSQL database.")
    
    except psycopg2.Error as e:
        print(f"Error uploading data: {e}")
        raise
    except ValueError as e:
        print(f"ValueError: {e}")
        raise
    finally:
        cursor.close()
        conn.close()


download()
extraction()
load()
upload()

File downloaded successfully: /home/ugoo/airflow/web-server-access-log.txt
Data extracted and transformed successfully: /home/ugoo/airflow/extracted-data.txt
Data loaded successfully into CSV: /home/ugoo/airflow/result.csv
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Skipping invalid row: []
Sk

In [3]:
#!pip install psycopg2